### Import Packages

In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize,ne_chunk,pos_tag
from nltk.util import ngrams
import itertools
import networkx as nx
from networkx.algorithms import community
% matplotlib notebook

### Detecting Files That Are Not Successfully Extracted

In [195]:
n = 0
for filename in filelist:
    if filename.endswith('txt'):

        print('extracting on file: ',filename)
        filepath = os.path.join(inputpath,filename)
        currentfile = pd.read_table(filepath,sep = '\t',index_col=False,encoding = 'utf-8')
        if currentfile.shape[0]<500:
            print('This file is abnormal->',filename,'\nNumber of records:',currentfile.shape[0])
            n+=1
print('There are altogather {} abnormal files.'.format(n))

extracting on file:  savedrecs (85).txt
This file is abnormal-> savedrecs (85).txt 
Number of records: 18
extracting on file:  savedrecs (1).txt
extracting on file:  savedrecs (50).txt
extracting on file:  savedrecs (46).txt
extracting on file:  savedrecs (11).txt
extracting on file:  savedrecs (66).txt
extracting on file:  savedrecs (31).txt
extracting on file:  savedrecs (27).txt
extracting on file:  savedrecs (70).txt
extracting on file:  savedrecs (71).txt
extracting on file:  savedrecs (26).txt
extracting on file:  savedrecs (30).txt
extracting on file:  savedrecs (67).txt
extracting on file:  savedrecs (10).txt
extracting on file:  savedrecs (47).txt
extracting on file:  savedrecs (51).txt
extracting on file:  savedrecs (84).txt
extracting on file:  savedrecs (60).txt
extracting on file:  savedrecs (37).txt
extracting on file:  savedrecs (21).txt
extracting on file:  savedrecs (76).txt
extracting on file:  savedrecs (83).txt
extracting on file:  savedrecs (7).txt
extracting on fi

ParserError: Error tokenizing data. C error: Expected 69 fields in line 6, saw 75


In [206]:
currentfile = pd.read_csv('savedrecs (68).csv')#,sep = '\t',index_col=False,header = None,error_bad_lines=False)

In [205]:
filepath = os.path.join(inputpath,'savedrecs (69).txt')
currentfile = pd.read_table(filepath,sep = '\t',index_col=False)

In [126]:
currentfile['PY'] = currentfile['PY'].astype(str)

In [132]:
currentfile['TC'] = currentfile['TC'].astype(str)

In [133]:
keep = ['AU','AF','TI','SO','OI','SN','PD','PY','VL','BP','EP','DI','UT','TC']
whole_column = set(currentfile.columns)
na_column = list(whole_column-set(keep))
currentfile.loc[:,na_column] = np.NaN

In [137]:
currentfile.to_csv(os.path.join(outputpath,filelist[0][:-4]+'_extract_cite_from_full'+filelist[0][-4:]),index = None, sep = '\t')

### Records Number Check

In [230]:
def check_by_year_records():
    
    all_data_store_path = '/Users/tylio/Google Drive/2018 spring/686/Wiley Sponsered Project/Data'
    current_data_folder_name = '(40000+)_by_year'
    current_data_store_path = os.path.join(all_data_store_path,current_data_folder_name)
    current_data_list = [f for f in os.listdir(current_data_store_path) if f.endswith('txt')]
    
    for data in current_data_list:
        
        print('Checking on: {}'.format(data))
        
        from_to = re.findall(r'-(\d+)',data)
        num_records = int(from_to[1])-int(from_to[0])+1
        try:
            current_df = pd.read_table(os.path.join(current_data_store_path,data),sep = '\t',index_col=False)
            if current_df.shape[0]!=num_records:
            
                print('Actual number of records is not in accordance with expected.\n')
                print('Expected: {}---Actual: {}'.format(num_records,current_df.shape[0]))
                
        except:
            print('Reading ERROR on {}'.format(data))
         

In [320]:
check_by_year_records()

Checking on: 2001-1-500.txt
Checking on: 2008-1001-1500.txt
Checking on: 2015-1-500.txt
Checking on: 2009-1501-2000.txt
Checking on: 2009-2001-2102.txt
Checking on: 1974-1-12.txt
Checking on: 2005-1001-1206.txt
Checking on: 1991-1-166.txt
Checking on: 2018-1-500.txt
Checking on: 2007-1-500.txt
Checking on: 2013-1-500.txt
Checking on: 1975-1-16.txt
Checking on: 2000-501-547.txt
Checking on: 1983-1-33.txt
Checking on: 1998-1-456.txt
Checking on: 1992-1-183.txt
Checking on: 1982-1-24.txt
Checking on: 2006-501-1000.txt
Checking on: 2016-1001-1500.txt
Checking on: 2014-501-1000.txt
Checking on: 2012-1001-1500.txt
Checking on: 2014-1-500.txt
Checking on: 2013-501-1000.txt
Checking on: 2002-501-729.txt
Checking on: 2000-1-500.txt
Checking on: 2014-2001-2500.txt
Checking on: 2016-2501-3000.txt
Checking on: 2013-1501-2000.txt
Checking on: 1973-1-7.txt
Checking on: 1978-1-17.txt
Checking on: 1972-1-14.txt
Checking on: 1977-1-22.txt
Checking on: 1994-1-282.txt
Checking on: 1993-1-224.txt
Checking

### Concatnate and Save

In [330]:
def concat_and_save():
    
    all_data_store_path = '/Users/tylio/Google Drive/2018 spring/686/Wiley Sponsered Project/Data'
    current_data_folder_name = '(40000+)_by_year'
    current_data_store_path = os.path.join(all_data_store_path,current_data_folder_name)
    current_data_list = [f for f in os.listdir(current_data_store_path) if f.endswith('txt')]
    whole_file = pd.DataFrame()
    
    for data in current_data_list:
        
        print('Loading on: {}'.format(data))
        
        from_to = re.findall(r'-(\d+)',data)
        num_records = int(from_to[1])-int(from_to[0])+1
        
        try:
            current_df = pd.read_table(os.path.join(current_data_store_path,data),sep = '\t',index_col=False)              
        except:
            print('Reading ERROR on {}'.format(data))
            
        assert current_df.shape[0]==num_records
        assert current_df.shape[1]==68
        
        whole_file = pd.concat([whole_file,current_df])
    whole_file.to_csv(os.path.join(current_data_store_path,'whole_file.csv'),index = False)
        
    return whole_file

In [331]:
whole_file = concat_and_save()

Loading on: 2001-1-500.txt
Loading on: 2008-1001-1500.txt
Loading on: 2015-1-500.txt
Loading on: 2009-1501-2000.txt
Loading on: 2009-2001-2102.txt
Loading on: 1974-1-12.txt
Loading on: 2005-1001-1206.txt
Loading on: 1991-1-166.txt
Loading on: 2018-1-500.txt
Loading on: 2007-1-500.txt
Loading on: 2013-1-500.txt
Loading on: 1975-1-16.txt
Loading on: 2000-501-547.txt
Loading on: 1983-1-33.txt
Loading on: 1998-1-456.txt
Loading on: 1992-1-183.txt
Loading on: 1982-1-24.txt
Loading on: 2006-501-1000.txt
Loading on: 2016-1001-1500.txt
Loading on: 2014-501-1000.txt
Loading on: 2012-1001-1500.txt
Loading on: 2014-1-500.txt
Loading on: 2013-501-1000.txt
Loading on: 2002-501-729.txt
Loading on: 2000-1-500.txt
Loading on: 2014-2001-2500.txt
Loading on: 2016-2501-3000.txt
Loading on: 2013-1501-2000.txt
Loading on: 1973-1-7.txt
Loading on: 1978-1-17.txt
Loading on: 1972-1-14.txt
Loading on: 1977-1-22.txt
Loading on: 1994-1-282.txt
Loading on: 1993-1-224.txt
Loading on: 2017-1501-2000.txt
Loading on: